In [2]:
import numpy as np
import pandas as pd

In [3]:
meta = pd.read_csv('polemo1_raw/meta.tsv', sep='\t')
meta

,domain,text,super,a,e,k
0,produkty,Mylący tytuł który sugeruje że mamy do czynien...,meta_minus_m,meta_minus_m,NaN,meta_minus_m
1,produkty,"Łóżeczko , jest śliczne wygląda bardzo masywan...",meta_plus_s,meta_amb,meta_amb,meta_plus_s
2,produkty,"Szczerze mówiąc , to telefon jest beznadziejny...",meta_minus_m,meta_minus_m,NaN,meta_minus_m
3,produkty,a ) Teleskop nadaje się najwyżej do obserwacji...,NaN,meta_minus_m,NaN,meta_minus_m
4,produkty,"Film , który zadowoli wszystkich fanów Kinga ....",meta_plus_s,meta_plus_m,meta_plus_s,meta_plus_s
...,...,...,...,...,...,...
9627,medycyna,Dziękujemy panu Doktorowi za naszego synka bo ...,meta_plus_m,meta_plus_m,NaN,meta_plus_m
9628,medycyna,W czasie wizyty spotkał em się z dużą życzliwo...,meta_plus_m,meta_plus_m,meta_plus_m,NaN
9629,medycyna,Jestem bardzo zadowolona z zabiegu biopsji mam...,meta_plus_s,meta_plus_m,meta_plus_s,NaN
9630,medycyna,witam : ) mój maż ma bardzo poważny problem z ...,meta_zero,meta_zero,meta_zero,NaN


In [4]:
sentences = pd.read_csv('polemo1_raw/zdania.tsv', sep='\t', usecols=['domain', 'text', 'super', 'a', 'e', 'k'])
sentences

/home/marcin/Documents/mgr/semestr_II/uczenie_reprezentacji/repr_venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,domain,text,super,a,e,k
0,produkty,"Dobrze wykonany , wygodny , zamki nie chodzą c...",z_plus_m,z_plus_m,NaN,z_plus_m
1,produkty,Stanowczo odradzam zakup wersji PL aż do momen...,z_minus_m,z_minus_m,NaN,z_minus_m
2,produkty,Ciezko mi sie odniesc do tego problemu .,z_amb,z_zero,NaN,z_amb
3,produkty,Brak dostępności tarcz w akceptowalnej cenie s...,z_minus_m,z_minus_m,NaN,z_minus_m
4,produkty,W moim subiektywnym odczuciu A4 Tech XL - 750B...,z_minus_m,z_minus_m,NaN,z_minus_m
...,...,...,...,...,...,...
66421,medycyna,ciemna maso a nie fotka u fotografa - a dlacze...,z_zero,z_zero,NaN,z_zero
66422,medycyna,"Wspomagali śmy zarówno tą większą , jak i mnie...",z_zero,z_zero,NaN,z_zero
66423,medycyna,Kompetencja oraz zaangażowanie Pana Doktora w ...,z_plus_m,z_plus_m,NaN,z_plus_m
66424,medycyna,"Jedna analiza wymagała 40 minut , a wynik był ...",z_zero,z_zero,NaN,z_zero


In [ ]:
from sklearn.metrics import cohen_kappa_score
def cohen_kappa_for_pair(first, second, df):
    both_not_na = df[~df[first].isna() & ~df[second].isna()]
    if both_not_na.shape[0] == 0:
        raise Exception('no pairs to count')
    return cohen_kappa_score(both_not_na[first], both_not_na[second]), both_not_na.shape[0]


def cohen_kappa_for_data(data, annotators):
    firsts = []
    seconds = []
    kappas = []
    sizes = []
    if not annotators:
        return pd.DataFrame()
    first = annotators[0]
    for second in annotators[1:]:
        kappa, size = cohen_kappa_for_pair(first, second, data)
        firsts.append(first)
        seconds.append(second)
        kappas.append(kappa)
        sizes.append(size)
    df = pd.DataFrame(data={'a': firsts, 'b': seconds, 'cohen_kappa_score': kappas, 'total_samples': sizes})
    return df.append(cohen_kappa_for_data(data, annotators[1:]), ignore_index=True)

In [30]:
display(cohen_kappa_for_data(meta, ['super', 'a', 'e', 'k']))
display(cohen_kappa_for_data(sentences, ['super', 'a', 'e', 'k']))

,a,b,cohen_kappa_score,total_samples
0,super,a,0.841610,9217.0
1,super,e,0.861916,4501.0
2,super,k,0.861805,5596.0
3,a,e,0.712314,4566.0
4,a,k,0.758025,5711.0
5,e,k,0.502300,978.0


,a,b,cohen_kappa_score,total_samples
0,super,a,0.842541,53877.0
1,super,e,0.914460,20776.0
2,super,k,0.916376,33021.0
3,a,e,0.787642,21580.0
4,a,k,0.751136,34177.0
5,e,k,1.000000,12.0


Zgodności z superanotatorem są silne. Zgodności między anotatorami wynoszące powyżej 0.7 nie są idealne, ale zwłaszcza w przypadku zdań są bliskie wartości 0.8, do której się dąży. Widać potrzebę korzystania z superanotatora, z którym wyniki są zadowalające. Zwraca uwagę bardzo niska zgodność między e i k w zbiorze meta. Anotatorzy mają najmniej wspólnie anotowanych próbek i zgadzają się w bardzo małej liczbie przykładów. Wyniki wskazują na dość dobrą jakość anotacji, jednak widać jeszcze, że istniejące procedury prawdopodobnie można poprawić.

In [58]:
!pip install simpledorff

In [ ]:
import simpledorff

In [73]:
krippendorff_meta = simpledorff.calculate_krippendorffs_alpha(meta.T.loc[['super', 'a', 'e', 'k']])
krippendorff_sentences = simpledorff.calculate_krippendorffs_alpha(sentences.T.loc[['super', 'a', 'e', 'k']])

In [78]:
print(f'krippendorff_alpha on meta     : {krippendorff_meta:.4f}')
print(f'krippendorff_alpha on sentences: {krippendorff_sentences:.4f}')

krippendorff_alpha on meta     : 0.8089
krippendorff_alpha on sentences: 0.8350


Użyto metryki nominal, jako że klasy są to oddzielne kategorie, wartości kategoryczne. Nie znając dokładnych zaleceń dla anotatorów porządkowanie wartości w rankingu na zasadzie: 'pozytywne > neutralne > negatywne' mogłoby nie być zgodne z intencją twórców danych i zakłamywać wyniki. Wartość metryki jest dość bliska do wartości uzyskanych z użyciem Cohen-Kappa, wskazując na ogólnie dość dobry wynik. Wartości są "uśrednione", więc nie widać wyższych wartości z superanotatorem i niższych między zwykłymi